In [278]:
import pandas as pd
import networkx as nx
import numpy as np
np.random.seed(357823)
import random
random.seed(357823)

In [279]:
edges=pd.read_csv('edges.csv')
vertices=pd.read_csv('nodes.csv')

In [280]:
edges.head()

,source,target
0,Baron Zemo,Blackout (Marcus Daniels)
1,Baron Zemo,Vermin (comics)
2,N'astirh,S'ym
3,N'astirh,Belasco (Marvel Comics)
4,N'astirh,Madelyne Pryor


In [281]:
vertices.head()

,group,id,size
0,1,Baron Zemo,2
1,1,N'astirh,3
2,0,Silver Sable,1
3,0,Hope Summers (comics),1
4,0,Magik (Illyana Rasputin),1


In [282]:
G=nx.from_pandas_edgelist(edges,source='source',target='target')

Métricas relacionales para cada nodo.

In [283]:
from collections import Counter
degree=nx.degree(G)
triangles=nx.triangles(G)
degree_centrality= nx.degree_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)
eigenvector_centrality = nx.eigenvector_centrality(G)
clustering = nx.clustering(G)
tam_comp = {}
maximo=max(len(c) for c in nx.connected_components(G))
for i, component in enumerate(nx.connected_components(G)):
    for node in component:
        tam_comp[node] = len(component)
tam=Counter((len(c) for c in nx.connected_components(G)))
vertices = vertices[vertices['id'].map(lambda v: tam_comp[v] >= 10)]
print(tam)



Counter({2: 30, 3: 11, 4: 5, 5: 4, 6: 3, 7: 1, 181: 1, 11: 1})


In [284]:
degree_dict= {d[0]:d[1] for d in degree}
degreeL = [degree[v["id"]]for _,v in vertices.iterrows()]
trianglesL=[triangles[v['id']] for _,v in vertices.iterrows()]
degree_centralityL = [degree_centrality[v['id']] for _,v in vertices.iterrows()]
closeness_centralityL = [closeness_centrality[v['id']] for _,v in vertices.iterrows()]
betweenness_centralityL = [betweenness_centrality[v['id']] for _,v in vertices.iterrows()]
eigenvector_centralityL =[eigenvector_centrality[v['id']] for _,v in vertices.iterrows()]
clusteringL = [clustering[v['id']] for _,v in vertices.iterrows()]
# component_idL= [component_id[v['id']] for _,v in vertices.iterrows()]

Crear el dataframe con los datos.

In [285]:
df = pd.DataFrame({'triangle':trianglesL, "degree_centrality":degree_centralityL,
                  'closeness_centrality':closeness_centralityL, 'betweenness_centrality':betweenness_centralityL,
                    'eigenvector_centrality':eigenvector_centralityL,'clustering':clusteringL,'group':vertices['group']})



In [286]:
df.head()

,triangle,degree_centrality,closeness_centrality,betweenness_centrality,eigenvector_centrality,clustering,group
3,0,0.002865,0.065194,0.000000,1.361871e-03,0.000000,0
6,0,0.002865,0.009551,0.000000,1.666953e-17,0.000000,1
9,0,0.017192,0.110388,0.176275,1.224383e-01,0.000000,1
14,0,0.002865,0.052361,0.000000,3.509777e-05,0.000000,2
16,1,0.008596,0.070706,0.011593,6.506930e-04,0.333333,1


In [287]:
atributos=df.iloc[:,0:-1]
objetivo=df.iloc[:,-1]

In [288]:
from sklearn.model_selection import train_test_split
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = train_test_split(
        # Conjuntos de datos a dividir, usando los mismos índices para ambos
        atributos, objetivo,
        # Tamaño del conjunto de prueba (20 % en este caso)
        test_size=.2,
        # Estratificación según la distribución de clases en el atributo objetivo
        stratify=objetivo,random_state=357823)

In [289]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

normalizador = ColumnTransformer(
    [
        ('normalizador', MinMaxScaler(), list(range(0, 6)))
    ]
)

In [290]:
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
rejilla_de_hiperparámetros = {
    # Máxima profundidad del árbol: 3, 4, 5, 6, 7, 8, 9, 10
    'arbol__max_depth': range(3, 11),
    # Mínimo número de ejemplos para poder particionar: 5, 10, 15
    'arbol__min_samples_split': range(2, 20)
}
kfold = StratifiedKFold(n_splits=10,random_state=357823,shuffle=True)
tuberia_arbol=Pipeline([('normalizador',normalizador),('arbol',DecisionTreeClassifier(random_state=357823))])
búsqueda_en_rejilla = GridSearchCV(tuberia_arbol,
                                   rejilla_de_hiperparámetros,
                                   scoring='accuracy',
                                   cv=kfold
                                   )
búsqueda_en_rejilla.fit(atributos_entrenamiento, objetivo_entrenamiento)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=357823, shuffle=True),
             estimator=Pipeline(steps=[('normalizador',
                                        ColumnTransformer(transformers=[('normalizador',
                                                                         MinMaxScaler(),
                                                                         [0, 1,
                                                                          2, 3,
                                                                          4,
                                                                          5])])),
                                       ('arbol',
                                        DecisionTreeClassifier(random_state=357823))]),
             param_grid={'arbol__max_depth': range(3, 11),
                         'arbol__min_samples_split': range(2, 20)},
             scoring='accuracy')

In [291]:
búsqueda_en_rejilla.cv_results_

{'mean_fit_time': array([0.00488071, 0.00461183, 0.00484242, 0.00464516, 0.00463097,
        0.00434303, 0.00456624, 0.00483532, 0.00530546, 0.00385997,
        0.00367231, 0.00407634, 0.00388095, 0.00435579, 0.00352743,
        0.00337441, 0.01047764, 0.00322134, 0.00377233, 0.00333569,
        0.00352809, 0.00361073, 0.00417943, 0.00446792, 0.00508199,
        0.00399218, 0.00480039, 0.00496802, 0.00563946, 0.00475268,
        0.00504634, 0.01315308, 0.00577514, 0.00606022, 0.00465243,
        0.0051105 , 0.00506997, 0.00506427, 0.00464134, 0.00562646,
        0.00454035, 0.00529578, 0.00498281, 0.00412464, 0.00428448,
        0.00386531, 0.0042208 , 0.00464492, 0.00468283, 0.00506015,
        0.00519781, 0.003707  , 0.00332646, 0.00478337, 0.00508134,
        0.00540478, 0.00422232, 0.00419316, 0.00432551, 0.00553689,
        0.0045011 , 0.00464265, 0.00378098, 0.00452759, 0.00499194,
        0.00370734, 0.00368009, 0.003356  , 0.00360103, 0.00404675,
        0.00362439, 0.00387106,

In [292]:
print(búsqueda_en_rejilla.best_score_)

0.7262500000000001


In [293]:

from sklearn.metrics import accuracy_score,classification_report
modelo_arbol=búsqueda_en_rejilla.best_estimator_
pred=modelo_arbol.predict(atributos_prueba)
print(accuracy_score(objetivo_prueba,modelo_arbol.predict(atributos_prueba)))
print(classification_report(objetivo_prueba,modelo_arbol.predict(atributos_prueba)))

0.717948717948718
              precision    recall  f1-score   support

           0       0.67      0.77      0.71        13
           1       0.81      0.77      0.79        22
           2       0.33      0.25      0.29         4

    accuracy                           0.72        39
   macro avg       0.60      0.60      0.60        39
weighted avg       0.71      0.72      0.71        39



In [339]:
from sklearn.neighbors import KNeighborsClassifier
rejilla_de_hiperparámetros = {
    # Máxima profundidad del árbol: 3, 4, 5, 6, 7, 8, 9, 10
    'knn__metric': ["euclidean","manhattan"],
    # Mínimo número de ejemplos para poder particionar: 5, 10, 15
    'knn__n_neighbors': range(1, 11)
}
kfold = StratifiedKFold(n_splits=10,random_state=357823,shuffle=True)
tuberia_arbol=Pipeline([('normalizador',normalizador),('knn',KNeighborsClassifier())])
búsqueda_en_rejilla = GridSearchCV(tuberia_arbol,
                                   rejilla_de_hiperparámetros,
                                   scoring='accuracy',
                                   cv=kfold)
búsqueda_en_rejilla.fit(atributos_entrenamiento, objetivo_entrenamiento)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=357823, shuffle=True),
             estimator=Pipeline(steps=[('normalizador',
                                        ColumnTransformer(transformers=[('normalizador',
                                                                         MinMaxScaler(),
                                                                         [0, 1,
                                                                          2, 3,
                                                                          4,
                                                                          5])])),
                                       ('knn', KNeighborsClassifier())]),
             param_grid={'knn__metric': ['euclidean', 'manhattan'],
                         'knn__n_neighbors': range(1, 11)},
             scoring='accuracy')

In [340]:
búsqueda_en_rejilla.best_score_

np.float64(0.6545833333333333)

In [341]:
modelo_knn=búsqueda_en_rejilla.best_estimator_
print(accuracy_score(objetivo_prueba,modelo_knn.predict(atributos_prueba)))
print(classification_report(objetivo_prueba,modelo_knn.predict(atributos_prueba)))

0.5897435897435898
              precision    recall  f1-score   support

           0       0.50      0.54      0.52        13
           1       0.78      0.64      0.70        22
           2       0.29      0.50      0.36         4

    accuracy                           0.59        39
   macro avg       0.52      0.56      0.53        39
weighted avg       0.63      0.59      0.61        39



In [317]:
from sklearn.preprocessing import OneHotEncoder
from keras.layers import Normalization,Dense
from keras import Sequential, Input
from sklearn.utils.class_weight import compute_class_weight
from keras.optimizers import Adam
import tensorflow as tf
import os
tf.random.set_seed(48679)
os.environ['PYTHONHASHSEED'] = '48679'
os.environ['TF_DETERMINISTIC_OPS'] = '1'

In [334]:
enc=OneHotEncoder()
objetivo_entrenamiento_cod=enc.fit_transform(objetivo_entrenamiento.to_frame()).toarray()
weights=compute_class_weight('balanced',
                     classes=np.array([0, 1,2]),
                     y=objetivo_entrenamiento)
print(weights)

red = Sequential()
normalizador_red=Normalization()
normalizador_red.adapt(atributos_entrenamiento.to_numpy())
red.add(Input(shape=(6,)))
red.add(normalizador_red)
red.add(Dense(50,activation="relu"))
red.add(Dense(10,activation="tanh"))
red.add(Dense(3,activation="softmax"))
red.compile(optimizer=Adam(learning_rate=0.05), loss='categorical_crossentropy',
                   metrics=['accuracy'])
red.fit(atributos_entrenamiento,objetivo_entrenamiento_cod,batch_size=256,epochs=300)

[1.         0.59302326 3.1875    ]
Epoch 1/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.2614 - loss: 1.1803
Epoch 2/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 0.5621 - loss: 1.1478
Epoch 3/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.5621 - loss: 0.9737
Epoch 4/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.5556 - loss: 0.8875
Epoch 5/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.4967 - loss: 0.9242
Epoch 6/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.4444 - loss: 0.9313
Epoch 7/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.4902 - loss: 0.8989
Epoch 8/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.5490 - loss: 0.8690
Epoch 9/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.5882 - loss: 0.8576
Epoch 10/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.5621 - loss: 0.8563
Epoch 11/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.5621 - loss: 0.8555
Epoch 12/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [335]:
pred=np.argmax(red.predict(atributos_prueba),axis=1)
print(classification_report(objetivo_prueba,pred))


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
              precision    recall  f1-score   support

           0       0.71      0.77      0.74        13
           1       0.86      0.86      0.86        22
           2       0.67      0.50      0.57         4

    accuracy                           0.79        39
   macro avg       0.75      0.71      0.73        39
weighted avg       0.79      0.79      0.79        39



In [336]:
from sklearn.preprocessing import KBinsDiscretizer
discretizador = ColumnTransformer([('discretizador',
                                    KBinsDiscretizer(
    n_bins=4, 
    encode='ordinal',  
    strategy='uniform'  
),
                                   list(range(0,6)))])

In [337]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import CategoricalNB
rejilla_de_hiperparámetros = {
    # Máxima profundidad del árbol: 3, 4, 5, 6, 7, 8, 9, 10
   
    # Mínimo número de ejemplos para poder particionar: 5, 10, 15
    'naive_bayes__alpha': range(1, 11)
    
}
kfold = StratifiedKFold(n_splits=5)
tuberia_arbol=Pipeline([('discretizador',discretizador),('naive_bayes',CategoricalNB())])
búsqueda_en_rejilla = GridSearchCV(tuberia_arbol,
                                   rejilla_de_hiperparámetros,
                                   scoring='accuracy',
                                   cv=kfold)
búsqueda_en_rejilla.fit(atributos_entrenamiento, objetivo_entrenamiento)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('discretizador',
                                        ColumnTransformer(transformers=[('discretizador',
                                                                         KBinsDiscretizer(encode='ordinal',
                                                                                          n_bins=4,
                                                                                          strategy='uniform'),
                                                                         [0, 1,
                                                                          2, 3,
                                                                          4,
                                                                          5])])),
                                       ('naive_bayes', CategoricalNB())]),
             param_grid={'naive_bayes__alpha': range(1, 11)},
             scoring='accuracy')

In [302]:
búsqueda_en_rejilla.best_score_

np.float64(0.5492473118279569)

In [338]:
modelo_arbol=búsqueda_en_rejilla.best_estimator_
pred=modelo_arbol.predict(atributos_prueba)
print(classification_report(objetivo_prueba,modelo_arbol.predict(atributos_prueba)))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        13
           1       0.55      0.95      0.70        22
           2       0.00      0.00      0.00         4

    accuracy                           0.54        39
   macro avg       0.18      0.32      0.23        39
weighted avg       0.31      0.54      0.39        39



c:\Users\alber\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\alber\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo